### 4.23 통계적 거리계산, 마하라노비스
* 데이터가 어떻게 퍼져있는지 알고싶을 때는 cv2.calcCovarMatrix
* 통계적 거리를 알고싶을 때는 cv2.Mahalanobis

In [10]:
# 0423.py
import cv2
import numpy as np

# 첫 번째 행 x 좌표, 두 번째 행 y 좌표 / 여기 코드에는 총 8개의 좌표를 나타냄
# (0, 0), (0, 50), (0, -50), (100, 0), 
# (100, 30), (150, 100), (-100, -20), (-150, -100) 
X = np.array([[0, 0,  0,100,100,150, -100,-150],
              [0,50,-50,  0, 30,100,  -20,-100]], dtype=np.float64)

X = X.transpose()
# X = X.T

# cov는 공분산 행렬, mean은 평균벡터
# 현재 X의 차원이 2차원 이라서 mean은 2개, cov는 2x2로 결과가 나타남
# mean 매개변수는 평균값을 기준으로 할지 설정
cov, mean = cv2.calcCovarMatrix(X, mean=None, 
                               flags = cv2.COVAR_NORMAL + cv2.COVAR_ROWS)
print('mean=', mean)
print('cov=', cov)

# 통계적 거리를 위한 연산

# 공분산 행렬의 역행렬 계산
ret, icov = cv2.invert(cov)
print('icov=',icov)

# 유클리드 거리 상으로는 50 차이
v1 = np.array([[0],[0]] , dtype=np.float64)
v2 = np.array([[0],[50]], dtype=np.float64)

# v1, v2의 통계적 거리 계산
# 통계적으로 둘이 얼마만큼 떨어져 있는지 확인
dist = cv2.Mahalanobis(v1, v2, icov)
print('dist = ', dist)
                
cv2.waitKey()    
cv2.destroyAllWindows()

mean= [[12.5   1.25  1.25]]
cov= [[73750.  34875.  34875. ]
 [34875.  26287.5 26287.5]
 [34875.  26287.5 26287.5]]
icov= [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]
dist =  0.0


### 4.24 4.23을 그림으로 표시한 것

In [11]:
# 0424.py
import cv2
import numpy as np
 
X = np.array([[0, 0,  0,100,100,150, -100,-150],
                 [0,50,-50,  0, 30,100,  -20,-100]], dtype=np.float64)
X = X.transpose() # X = X.T

cov, mean = cv2.calcCovarMatrix(X, mean=None,
                                    flags=cv2.COVAR_NORMAL+cv2.COVAR_ROWS)
ret, icov = cv2.invert(cov)

dst = np.full((512,512,3), (255, 255, 255), dtype= np.uint8)
rows, cols, channel = dst.shape
centerX = cols//2
centerY = rows//2

v2 = np.zeros((1,2), dtype=np.float64)
FLIP_Y = lambda y: rows - 1 - y

# 통계적 거리에 따라 같은 거리를 같은 색으로 표시
for y in range(rows):
    for x in range(cols):
        v2[0,0] = x - centerX
        v2[0,1] = FLIP_Y(y) - centerY # y-축 뒤집기
        dist = cv2.Mahalanobis(mean, v2, icov)
        if dist < 0.1:
            dst[y, x] = [50, 50, 50]
        elif dist < 0.3:
            dst[y, x] = [100, 100, 100]
        elif dist < 0.8:
            dst[y, x] = [200, 200, 200]
        else:
            dst[y, x] = [250, 250, 250]

# 좌표 점 표시
for k in range(X.shape[0]):
    x, y = X[k,:]
    cx = int(x+centerX)
    cy = int(y+centerY)
    cy = FLIP_Y(cy)
    cv2.circle(dst,(cx,cy),radius=5,color=(0,0,255),thickness=-1)
    
# draw X, Y-axes
cv2.line(dst, (0, 256), (cols-1, 256), (0, 0, 0))
cv2.line(dst, (256,0), (256,rows), (0, 0, 0))

# 공분산 행렬을 통해 고유 값과 고유벡터 출력
ret, eVals, eVects = cv2.eigen(cov)
print('eVals=',  eVals)
print('eVects=', eVects)

def ptsEigenVector(eVal, eVect):
##    global mX, centerX, centerY
    scale = np.sqrt(eVal) # eVal[0]
    x1 = scale*eVect[0]
    y1 = scale*eVect[1]
    x2, y2 = -x1, -y1 # 대칭

    x1 += mean[0,0] + centerX
    y1 += mean[0,1] + centerY
    x2 += mean[0,0] + centerX
    y2 += mean[0,1] + centerY
    y1 = FLIP_Y(y1)
    y2 = FLIP_Y(y2)
    return int(x1), int(y1), int(x2), int(y2)

# 고유값과 고유 벡터를 통해 직선 표시
# draw eVects[0]
x1, y1, x2, y2 = ptsEigenVector(eVals[0], eVects[0])
cv2.line(dst, (x1, y1), (x2, y2), (255, 0, 0), 2)

# draw eVects[1]
x1, y1, x2, y2 = ptsEigenVector(eVals[1], eVects[1])
cv2.line(dst, (x1, y1), (x2, y2), (255, 0, 0), 2)

cv2.imshow('dst', dst)               
cv2.waitKey()    
cv2.destroyAllWindows()

eVals= [[92202.13359547]
 [ 7835.36640453]]
eVects= [[ 0.88390424  0.46766793]
 [-0.46766793  0.88390424]]


### 4.25 PCA 투영, 역투영

In [13]:
# 0425.py
import cv2
import numpy as np

X = np.array([[0, 0,  0,100,100,150, -100,-150],
                 [0,50,-50,  0, 30,100,  -20,-100]], dtype=np.float64)
X = X.transpose() # X = X.T

##mean = cv2.reduce(X, 0, cv2.REDUCE_AVG)
##print('mean = ', mean)

# 평균벡터와 고유벡터 추출
mean, eVects = cv2.PCACompute(X, mean=None)
print('mean = ', mean)
print('eVects = ', eVects)

# PCA 투영
Y =cv2.PCAProject(X, mean, eVects)
print('Y = ', Y)

# PCA 역투영
X2 = cv2.PCABackProject(Y, mean, eVects)
print('X2 = ', X2)
print(np.allclose(X, X2)) # X와 X2의 값 비교 같을 시 True 반환
cv2.waitKey()    
cv2.destroyAllWindows()

mean =  [[12.5   1.25]]
eVects =  [[ 0.88390424  0.46766793]
 [-0.46766793  0.88390424]]
Y =  [[ -11.63338792    4.74096885]
 [  11.75000868   48.93618085]
 [ -35.01678451  -39.45424315]
 [  76.75703609  -42.02582434]
 [  90.78707404  -15.50869713]
 [ 167.71904127   22.98120308]
 [-109.37717055   33.82967723]
 [-190.9858171   -13.49926538]]
X2 =  [[ 1.77635684e-15  0.00000000e+00]
 [ 3.55271368e-15  5.00000000e+01]
 [ 0.00000000e+00 -5.00000000e+01]
 [ 1.00000000e+02 -7.10542736e-15]
 [ 1.00000000e+02  3.00000000e+01]
 [ 1.50000000e+02  1.00000000e+02]
 [-1.00000000e+02 -2.00000000e+01]
 [-1.50000000e+02 -1.00000000e+02]]
X =  [[   0.    0.]
 [   0.   50.]
 [   0.  -50.]
 [ 100.    0.]
 [ 100.   30.]
 [ 150.  100.]
 [-100.  -20.]
 [-150. -100.]]
True


### 4.26 3채널 컬러 영상의 PCA 투영

In [18]:
# 0426.py
import cv2
import numpy as np

src = cv2.imread('c:/data/lena.jpg')
src= cv2.resize(src, (300, 300))
b, g, r = cv2.split(src) 
cv2.imshow('b', b)
cv2.imshow('g', g)
cv2.imshow('r', r)

X = src.reshape(-1, 3)
print('X.shape=', X.shape) # 첫번째 값이 픽셀 개수

# PCA(주 성분 분석)
# 입력 영상의 차원이 3개 이므로 3개의 평균벡터, 3x3 고유벡터 추출
mean, eVects = cv2.PCACompute(X, mean=None)
print('mean = ', mean)
print('eVects = ', eVects)

# PCA 투영
Y = cv2.PCAProject(X, mean, eVects) # 평균벡터, 고유벡터 입력
Y = Y.reshape(src.shape) # opencv 상에서 보여주기 위해 원본과 똑같이 shape
print('Y.shape=', Y.shape)

eImage = list(cv2.split(Y)) # 채널 분리
for i in range(3):
    cv2.normalize(eImage[i], eImage[i], 0, 255, cv2.NORM_MINMAX)
    eImage[i]=eImage[i].astype(np.uint8)
    
cv2.imshow('eImage[0]', eImage[0])
cv2.imshow('eImage[1]', eImage[1])
cv2.imshow('eImage[2]', eImage[2])
cv2.waitKey()    
cv2.destroyAllWindows()

X.shape= (90000, 3)
mean =  [[105.272224  99.431435 179.60574 ]]
eVects =  [[ 0.39347526  0.6897963   0.6077484 ]
 [-0.6337587  -0.27536684  0.7228575 ]
 [ 0.6659781  -0.6695924   0.32881448]]
Y.shape= (300, 300, 3)
